# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,-4.71,62,100,11.61,SJ,1706721282
1,1,rongelap,11.1635,166.8941,26.12,81,100,10.63,MH,1706721303
2,2,port-aux-francais,-49.3500,70.2167,4.27,74,97,9.78,TF,1706721185
3,3,iqaluit,63.7506,-68.5145,-33.15,62,20,8.23,CA,1706720915
4,4,georgetown,5.4112,100.3354,27.96,83,20,1.03,MY,1706720941


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

humidity_level = city_data_df['Humidity']

# Configure the map plot
original_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)
# Display the map
original_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

#Maximum Temps below 27 but above 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind Speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#No clouds
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
                                
# Drop any rows with null values
city_data_df = city_data_df.dropna()                         

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,pisco,-13.7000,-76.2167,26.68,73,0,4.12,PE,1706721616
344,344,mae ramat,16.9840,98.5166,22.11,76,0,0.79,TH,1706721747
462,462,reggane,26.7158,0.1714,22.44,19,0,3.63,DZ,1706721808


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Select the desired columns from the city_data_df DataFrame
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
95,pisco,PE,-13.7000,-76.2167,73,
344,mae ramat,TH,16.9840,98.5166,76,
462,reggane,DZ,26.7158,0.1714,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =  f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
mae ramat - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
95,pisco,PE,-13.7000,-76.2167,73,La Portada
344,mae ramat,TH,16.9840,98.5166,76,No hotel found
462,reggane,DZ,26.7158,0.1714,19,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Set parameters to search for a hotel
    radius = 10000
    params = {
        "categories": "accommodation.hotel",
        "apiKey": geoapify_key
    }

    # Print a message to follow up the hotel search
    print("Starting hotel search")

    # Iterate through the hotel_df DataFrame
    for index, row in hotel_df.iterrows():
        # get latitude, longitude from the DataFrame
        latitude = row['Lat']
        longitude = row["Lng"]

        # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
        params["filter"] = f"circle:{longitude},{latitude},{radius}"
        params["bias"] =  f"proximity:{longitude},{latitude}"

        # Set base URL
        base_url = "https://api.geoapify.com/v2/places"

        # Make an API request using the params dictionary
        name_address = requests.get(base_url, params=params)

        # Convert the API response to JSON format
        name_address = name_address.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

        # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

    # Display sample data 
    hotel_df

IndentationError: unexpected indent (3764383594.py, line 2)

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)